# Data structure
* tensorflow program = data structure of tensor + algorithm in graph
* 2 type of tensor: constant and variable
* variable can be re-assigned through operator "assign"
* Từng kiểu dữ liệu được biểu diễn bởi tensor với các rank khác nhau (array là rank=1, matrix rank=2, image rank=3)

## Constant tensor

In [18]:
import numpy as np
import tensorflow as tf
i = tf.constant(1) # tf.int32
l = tf.constant(1, dtype = tf.int64)
f = tf.constant(1.23)
d = tf.constant(3.14, dtype=tf.double)
s = tf.constant("hello world")
b = tf.constant(True)

print(l)
print(tf.double == np.float64)
print(tf.string == np.unicode)

tf.Tensor(1, shape=(), dtype=int64)
True
False


In [8]:
scalar = tf.constant(1)
print(tf.rank(scalar))
print(scalar.numpy().ndim)

tf.Tensor(0, shape=(), dtype=int32)
0


In [10]:
vector = tf.constant([1.0, 2.0, 3.0, 4.0])
print(tf.rank(vector))
print(np.ndim(vector.numpy()))

tf.Tensor(1, shape=(), dtype=int32)
1


In [11]:
matrix = tf.constant([[1.0,2.0],[3.0,4.0]])
print(tf.rank(matrix).numpy())
print(np.ndim(matrix))

2
2


* Change data type of tensor: tf.cast(data , type)
* Check size tensor: method .shape

In [17]:
h = tf.constant([[123, 456], [1,2]], dtype=tf.int32)
f = tf.cast(h, tf.float32)
print(h.dtype, f.dtype)
print(h.shape)

<dtype: 'int32'> <dtype: 'float32'>
(2, 2)


In [14]:
u = tf.constant(u"hello world")
print(u.numpy())
print(u.numpy().decode("utf-8"))

b'hello world'
hello world


## Variable tensor
* using tf.Variable(data, name="name")

In [20]:
# reassign create new space variable
c = tf.constant([1.0, 2.0])
print(id(c))
c = c + tf.constant([1.0, 1.0])
print(c)
print(id(c))

140608006159440
tf.Tensor([2. 3.], shape=(2,), dtype=float32)
140607963991888


In [21]:
v = tf.Variable([1.0, 2.0], name='v')
print(id(v))
v.assign_add([1.0, 1.0])
print(v)
print(id(v))

140607963561488
<tf.Variable 'v:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
140607963561488


# Type of graph
* three type graph: static, dynamic and Autograph
* dynamic graph (Eager execution): added to invisible graph and executed after its usage
* @tf.function convert code to static tensorflow graph, call Autograph 
* graph = nodes (operator) + edges (dependencies) 

## static graph

In [25]:
import tensorflow as tf

g = tf.compat.v1.Graph()
# define graph
with g.as_default():
    x = tf.compat.v1.placeholder(name='x', shape=[], dtype=tf.string)
    y = tf.compat.v1.placeholder(name='y', shape=[], dtype=tf.string)
    z = tf.strings.join([x, y], name="join", separator=' ')
# enecuting it in session
with tf.compat.v1.Session(graph = g) as sess:
    result = sess.run(fetches = z, feed_dict = {x: "hello", y:"world"})
    print(result)

b'hello world'


## Dynamic graph
* lower efficiency in execution

In [73]:
@tf.function
def strjoin(x, y):
    z = tf.strings.join([x, y], separator=' ')
    tf.print(z)
    return z

x = tf.constant("hello")
y = tf.constant("world")
strjoin(x, y)

hello world


<tf.Tensor: shape=(), dtype=string, numpy=b'hello world'>

## Autograph

In [74]:
import datetime

import os
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('data', 'autograph', stamp)

from pathlib import Path
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = str(Path('./data/autograph/' + stamp))

writer = tf.summary.create_file_writer(logdir)

# Start tracing on Autograph
tf.summary.trace_on(graph=True, profiler=True)

# Execute Autograph
resutl =  strjoin("hello", "world")

# write into log
with writer.as_default():
    tf.summary.trace_export(
        name="autograph",
        step=0,
        profiler_outdir=logdir
    )

hello world


In [75]:
# magic comment run tfboard in jupyter
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [76]:
# launch tensorbroard
%tensorboard --logdir ./data/autograph/

Reusing TensorBoard on port 6006 (pid 22333), started 0:03:17 ago. (Use '!kill 22333' to kill it.)

# Automatic Differentiate
* using tf.GradientTape to record forward calculation and reverse

In [77]:
import tensorflow as tf
import numpy as np

x = tf.Variable(0.0, name="x", dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = a * tf.pow(x, 2) + b * x + c

dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(-2.0, shape=(), dtype=float32)


In [78]:
# Calculate the second order derivative
with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:   
        y = a*tf.pow(x,2) + b*x + c
    dy_dx = tape1.gradient(y,x)   
dy2_dx2 = tape2.gradient(dy_dx,x)

print(dy2_dx2)

tf.Tensor(2.0, shape=(), dtype=float32)


In [79]:
# Use it in the autograph

@tf.function
def f(x):   
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    
    # Convert the type of the variable to tf.float32
    x = tf.cast(x,tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = a*tf.pow(x,2)+b*x+c
    dy_dx = tape.gradient(y,x) 
    
    return((dy_dx,y))

tf.print(f(tf.constant(0.0)))
tf.print(f(tf.constant(1.0)))

(-2, 1)
(0, 0)
